# Linux私房手册

所有测试，代码使用的环境是CentOS7.0，不同版本的Linux可能会略有不同。

## 常见问题收集

### source filename 与 sh filename 及./filename执行脚本的区别?

当shell脚本具有可执行权限时，用sh filename与./filename执行脚本是没有区别得。./filename是因为当前目录没有在PATH中，所有”.”是用来表示当前目录的。sh filename 重新建立一个子shell，在子shell中执行脚本里面的语句，该子shell继承父shell的环境变量，但子shell新建的、改变的变量不会被带回父shell，除非使用export。

`source filename`只是简单地读取脚本里面的语句依次在当前shell里面执行，没有建立新的子shell。那么脚本里面所有新建、改变变量的语句都会保存在当前shell里面。

### 如何配置sftp服务

[Linux（CentOS）上配置 SFTP](https://www.cnblogs.com/whatmiss/p/7068772.html)

这里有几个坑需要注意：
1. 网上几乎所有的教程都会说要向下面这样修改`sshd_confg`配置文件：
```
Match User sftpuser #这行用来匹配用户
ChrootDirectory /datas/www  #用chroot将用户的根目录指定到/datas/www ，这样用户就只能在/datas/www下活动
AllowTcpForwarding no
ForceCommand internal-sftp #指定sftp命令
```

但是，在`Red Hat Enterprise Linux Server release 7.0`版本下，`Match User sftpuser`后面的几行要缩进一个`tab`制表符，不然不生效。

2. 不能连接，总是有如下的错误：
```
> Write failed: Broken pipe     
> Couldn't read packet: Connection reset by peer
```
这个是因为从设置的用户的`ftp`根目录向上一直到系统根目录，所有的文件夹的目录拥有者必须是`root`，组可以不是`root`，且所有文件夹的权限必须是755或者750，即不可以具有群组写入权限。如果要有写权限怎么办？目前能想到的是在用户根目录下新建其它用户的目录，并赋予写权限，但此时根目录仍然没有写权限，还没有查到相应得解决方案。

### 使用`crontab`运行python脚本?

[Linux-Centos用crontab定时运行python脚本详细步骤](https://www.cnblogs.com/qq874455953/p/9589207.html)

`crontab`默认的路径是当前用户的`home`目录，因此如果脚本不在当前用户的`home`目录的话，不管是`crontab`命令本身，还是要执行的脚本里面的路径，都需要使用绝对路径。比如，脚本经常会这样写：
```python
f.open("test.txt", 'w')
f.close()
```
表示在脚本的当前目录下打开一个"test.txt"的文件，此时很可能失效，因为系统会在当前登录用户的`home`目录下生成"test.txt"，而不是在和脚本文件相同的目录下。

### `nohub`和`&`有什么区别？

- [nohub,setid和&的区别](https://zhidao.baidu.com/question/460756573.html)
- [linux中nohup和&有什么区别](https://zhidao.baidu.com/question/2057683352444428427.html?qbl=relate_question_0)

这两个命令一般组合起来使用，其中：
- nohub：远程登录系统，其实是一个开启了一个shell bash进程，所有运行的命令都是该进程的子进程。而将终端关闭的时候，会发送一个hangup信号来关闭子进程。使用nohub则会捕获并且忽略这个信号，因此即使终端关闭了，子进程也不会关闭。同时他还将子进程的输出重定位到nohup.out文件，这样即使子进程中有打印语句，因为已经重定向到文件中，也不会报错。
- &：表示将进程放入后台运行，正常情况下，关闭终端会结束子进程，可以通过`shopt | grep huponexit`查看shell是否将SIGHUP发送给它的子进程，如果为false的话，关闭终端也不会结束子进程。但是此时子进程的stdout标准输出和stderr标准错误会丢失，因此如果脚本里面包含print语句，会抛出错误。

### `<<`，`<<<`和`< <`的区别

- [stackoverflow](https://askubuntu.com/questions/678915/whats-the-difference-between-and-in-bash)

- `<<`被称为Here document，`<<`后面的多行被当作标准输入，`<<`后面可以接一个任意单词作为文档的结束符，比如：
 ```shell
 $ wc << EOF
 > one two three
 > four five
 > EOF
 2  5 24
 ```
- `<<<`被成为Here string，表示后面接着的是一个字符串。
- `< <`实际上可以把它看成是两步，比如：
 ```shell
 $ wc < <(echo bar;echo foo)
  2       2       8
 ```
 可以看成先执行`echo bar`和`echo foo`，通过`<`生成一个临时的`pipe`文件，然后将该文件的内容作为标准输入发送给`wc`命令。虽然效果和`eecho bar;echo foo) | wc`是一样的，但是在底层实现完全不同。某些场景下，只能使用进程替换，无法使用管道符，比如`diff <(ls /bin) <(ls /usr/bin)`。


### 如何批量的关闭掉多个进程？

可以使用如下的指令组合：  
`ps aux | grep chrprocesser(这里写进程名称) | grep -v grep | awk '{print $2}' | xargs kill -9`

### 单引号、双引号、不加引号和反引号的区别

1. 不加引号：不会将含有空格的字符串视为一个整体输出, 如果内容中有变量等，会先把变量解析出结果，然后再输出最终内容来，如果字符串中带有空格等特殊字符，则不能完整的输出，需要改加双引号，一般连续的字符串，数字，路径等可以用。
2. 单引号：可以说是所见即所得：即将单引号内的内容原样输出，或者描述为单引号里面看到的是什么就会输出什么。
3. 双引号：把双引号内的内容输出出来；如果内容中有变量，会先把变量解析出结果，然后在输出最终内容来。单引号和双引号之分，二者的主要区别在于，被单引号括起来的字符都是普通字符，就算特殊字符也不再有特殊含义；而被双引号括起来的字符中，`$`、`\`和反引号是拥有特殊含义的，`$`代表引用变量的值，而反引号代表引用命令。
4. 反引号：如果需要调用命令的输出，或把命令的输出赋予变量，则命令必须使用反引号包含，这条命令才会执行。反引号的作用和 `$(命令)` 是一样的，但是反引号非常容易和单引号搞混，所以推荐大家使用 `$(命令)` 的方式引用命令的输出。

### 如何正确的设置时区

使用`date -R`发现当前时区不对。
1. 方法1：   
查看`/etc/localtime`文件，发现链接的文件不对，执行指令：
```
ln -sf /usr/share/zoneinfo/Asia/Shanghai /etc/localtime
```
直接覆盖也可，这种方法需要重启电脑。


2. 方法2：  
直接编辑`/etc/profile`文件，在最后添加`TZ='CST-8'`，然后`source /etc/profile`即可，这种方法可以不用重启。

## 教程收集

### shell指令

#### find

- [Linux下find指令](https://www.cnblogs.com/guochuanrui/p/5515158.html) 简单易懂
- [find常用参数详解](https://www.cnblogs.com/yinzhengjie/p/6207318.html) 讲解比较全面，但是不够细致
- [Linux中find命令用法大全](https://www.cnblogs.com/Ido-911/p/9638612.html) 主要讲解查找文件的5种方式，例子较多

这里补充一下`-exec`的用法，`exec`参数表示对查找出来的文件执行`exec`后面跟着的shell指令，比如：
```bash
find ./ -type f -exec ls -l {} \;
```
`{}`代替`find`搜索到的文件名，`\;`对分号进行转义，输出为：
```bash
-rwxrwxr-x. 1 danni danni 939 Sep 20 18:52 ./test.sh
-rw-rw-r--. 1 danni danni 556 Oct 13 22:48 ./test.txt
```

比如：
1. 搜索名字为`index.html`的文件并且改名为`index.html.2019-09-23`的样式:
```
find /etc/ /tmp/ /root/ -name "index.html" -exec mv {} {}.`date +%F` \;
```
2. 搜索30天以前的文件并且删除：
```
find /tmp/ -atime +30 -exec rm -rf {} \;
```

#### date

`date`命令用来显示系统时间，后面可以加参数进行格式化：
- [Linux下date命令](https://blog.csdn.net/jk110333/article/details/8590746)

比如：
1. `date "+%Y-%m-%d"`，输出`2013-02-19`。
2. `date "+%Y-%-m-%d"`，输出`2013-2-19`，其中`%-m`表示月份前面不补0。
3. `date -d tomorrow`，输出`2019-9-23`。

#### 其它

- [xargs命令详解，xargs与管道的区别](https://www.cnblogs.com/wangqiguo/p/6464234.html)
- [ps和top以及ps aux和ps -ef的区别](https://www.cnblogs.com/tudachui/p/9516009.html)
- [awk程序设计语言](https://awk.readthedocs.io/en/latest/chapter-one.html)

### 进程管理

- [如何使用supervisor](https://csjourney.com/managing-processes-with-supervisor-in-depth-tutorial/)